In [21]:
import pandas as pd
teams = pd.read_csv('happy.csv')

In [22]:
teams

,team_id,team_name,team_url,conference_id,conference,division,year,season_id
0,NaN,Boston College,/teams/596496,821,ACC,1,2025,NaN
1,NaN,California,/teams/596511,821,ACC,1,2025,NaN
2,NaN,Clemson,/teams/596523,821,ACC,1,2025,NaN
3,NaN,Duke,/teams/596708,821,ACC,1,2025,NaN
4,NaN,Florida St.,/teams/596545,821,ACC,1,2025,NaN
...,...,...,...,...,...,...,...,...
939,NaN,Wis.-Oshkosh,/teams/597082,1473,WIAC,3,2025,NaN
940,NaN,Wis.-Platteville,/teams/597083,1473,WIAC,3,2025,NaN
941,NaN,Wis.-Stevens Point,/teams/597084,1473,WIAC,3,2025,NaN
942,NaN,Wis.-Stout,/teams/597085,1473,WIAC,3,2025,NaN


In [23]:
import pandas as pd
import requests

# Sample DataFrame with team URLs
df = teams
base_url = "https://stats.ncaa.org"
stat_categories = {"pitching": "15688", "batting": "15687"}

# Function to get HTML content with custom headers
def get_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch {url}, Status code: {response.status_code}")
        return None
    return response.text

# Dictionary to store HTML content for all teams and categories
team_htmls = {}

for _, row in df.iterrows():
    team_url = row['team_url']
    
    for stat_type, stat_id in stat_categories.items():
        full_url = f"{base_url}{team_url}/season_to_date_stats?year_stat_category_id={stat_id}"
        html_content = get_html(full_url)
        
        if html_content:
            # Store HTML content with team URL and stat type
            team_htmls[(team_url, stat_type)] = html_content

# Now 'team_htmls' contains HTML content for each team and stat category
print(f"Fetched HTML for {len(team_htmls)} combinations.")


Fetched HTML for 1888 combinations.


In [24]:
import pandas as pd
from bs4 import BeautifulSoup

# Function to extract table from HTML content
def extract_table(html_content, stat_type, team_url, team_name):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find("table")  # You can adjust the table selector if necessary
    if not table:
        print(f"No table found for {team_url} - {stat_type}")
        return None

    df = pd.read_html(str(table))[0]  # Convert the table to a DataFrame
    df['team_name'] = team_name  # Add team name column directly from the input
    df['stat_type'] = stat_type  # Add a stat_type column to distinguish between pitching and hitting
    return df

# Initialize lists to store data for pitching and hitting separately
pitching_data = []
hitting_data = []

# Assuming 'df' contains the team_url and team_name columns
for _, row in df.iterrows():
    team_url = row['team_url']
    team_name = row['team_name']
    
    for stat_type, stat_id in stat_categories.items():
        full_url = f"{base_url}{team_url}/season_to_date_stats?year_stat_category_id={stat_id}"
        html_content = team_htmls.get((team_url, stat_type))
        
        if html_content:
            # Extract the table for each stat type (pitching or batting)
            table_df = extract_table(html_content, stat_type, team_url, team_name)
            if table_df is not None:
                if stat_type == 'pitching':
                    pitching_data.append(table_df)
                elif stat_type == 'batting':
                    hitting_data.append(table_df)

# Combine all pitching and hitting data into two separate DataFrames
pitching_df = pd.concat(pitching_data, ignore_index=True)
hitting_df = pd.concat(hitting_data, ignore_index=True)

# Preview the final dataframes
print("Pitching DataFrame:")
print(pitching_df.head())
print("\nHitting DataFrame:")
print(hitting_df.head())


Pitching DataFrame:
                                 #          Player   Yr  Pos   Ht  B/T  App  \
0                               49  Peter Schaefer   Sr    P  6-0  L/R  1.0   
1  Succ-Opp against Leadoff Batter             NaN  NaN  NaN  NaN  NaN  NaN   
2          Hits-AB with runners ob             NaN  NaN  NaN  NaN  NaN  NaN   
3                   Hits-AB vs lhb             NaN  NaN  NaN  NaN  NaN  NaN   
4                   Hits-AB w2outs             NaN  NaN  NaN  NaN  NaN  NaN   

   GS  ERA   IP  ...    0    1    2    3    4    5    6    7    8    9  
0 NaN  0.0  1.1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1 NaN  0.0  0.1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2 NaN  0.0  0.1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3 NaN  0.0  0.1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4 NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 50 columns]

Hitting DataFrame:
                        

In [25]:
pitching_df

,#,Player,Yr,Pos,Ht,B/T,App,GS,ERA,IP,...,0,1,2,3,4,5,6,7,8,9
0,49,Peter Schaefer,Sr,P,6-0,L/R,1.0,NaN,0.0,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Succ-Opp against Leadoff Batter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hits-AB with runners ob,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hits-AB vs lhb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hits-AB w2outs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...",NaN,NaN
74489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM
74490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Brevard (12-9),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Greensboro (4-19),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
hitting_df

,#,Player,Yr,Pos,Ht,B/T,GP,GS,BA,OBPct,...,0,1,2,3,4,5,6,7,8,9
0,15,Jace Roossien,Fr,C,6-0,B/R,1,0,NaN,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hits-AB vs Right Handed Pitchers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Succ-Opp as leadoff batter in inning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hits-AB bases empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38,Jack Toomey,Jr,OF,6-3,R/R,2,2,0.571,0.556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...","Live Box Score $(""#livestream_box_score_58038...",NaN,NaN
108744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM,02/18/2025 05:30 PM
108745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Brevard (12-9),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Greensboro (4-19),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
hitting_df.to_csv('hitting10.csv')
pitching_df.to_csv('pitching10.csv')

In [28]:
# For pitching_df
pitching_df = pitching_df[
    pitching_df['Player'].notna() &
    ~pitching_df['Player'].str.lower().isin(["totals", "opponent totals"])
]

# For hitting_df
hitting_df = hitting_df[
    hitting_df['Player'].notna() &
    ~hitting_df['Player'].str.lower().isin(["totals", "opponent totals"])
]
pitching_df

,#,Player,Yr,Pos,Ht,B/T,App,GS,ERA,IP,...,0,1,2,3,4,5,6,7,8,9
0,49,Peter Schaefer,Sr,P,6-0,L/R,1.0,NaN,0.00,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,37,John Kwiatkowski,Jr,P,6-6,L/L,1.0,NaN,13.50,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,30,JD Ogden,Sr,P,6-5,B/L,1.0,NaN,81.00,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,50,Joey Ryan,Sr,P,6-5,L/R,1.0,NaN,0.00,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,47,Matthew Spada,Jr,P,6-1,L/L,1.0,NaN,0.00,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74425,32,Maxwell Kasprowisz,Sr,P,6-0,-/L,1.0,NaN,18.00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74434,35,Jarrett Hall,Sr,P,5-10,-/R,1.0,1.0,23.63,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74443,2,Eddie Sanchez,Jr,SS,5-10,NaN,1.0,NaN,13.50,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74452,52,Landon Coleman,Fr,P,6-1,-/R,1.0,NaN,0.00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
UVNROSTER = pd.read_csv('NIGHTHAWKSROSTER.csv')
UVNROSTER

,Player,Pos,School,Player_School
0,Will Harrigan,SP,Columbia,Will HarriganColumbia
1,Brendan Walker,SP,Stetson,Brendan WalkerStetson
2,Jack Ruditzky,SP,Rider,Jack RuditzkyRider
3,JP Podgorski,SP,Villanova,JP PodgorskiVillanova
4,Marco Dumsky,SP,Dartmouth,Marco DumskyDartmouth
5,Owen Schexnaydre,SP,Louisiana Tech,Owen SchexnaydreLouisiana Tech
6,Nate Ross,SP,Central Michigan,Nate RossCentral Michigan
7,Steven Mazza,SP,Marist,Steven MazzaMarist
8,Brendan Callanan,SP,Iona,Brendan CallananIona
9,Nick Tamburro,SP,Monmouth,Nick TamburroMonmouth


In [30]:
# Ensure both columns are treated as strings and then concatenate without a space
pitching_df['Player_School'] = pitching_df['Player'].astype(str) + pitching_df['team_name'].astype(str)
hitting_df['Player_School'] = hitting_df['Player'].astype(str) + hitting_df['team_name'].astype(str)
pitching_df

C:\Users\hayde\AppData\Local\Temp\ipykernel_28688\3923758945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitching_df['Player_School'] = pitching_df['Player'].astype(str) + pitching_df['team_name'].astype(str)
C:\Users\hayde\AppData\Local\Temp\ipykernel_28688\3923758945.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hitting_df['Player_School'] = hitting_df['Player'].astype(str) + hitting_df['team_name'].astype(str)


,#,Player,Yr,Pos,Ht,B/T,App,GS,ERA,IP,...,1,2,3,4,5,6,7,8,9,Player_School
0,49,Peter Schaefer,Sr,P,6-0,L/R,1.0,NaN,0.00,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Peter SchaeferBoston College
10,37,John Kwiatkowski,Jr,P,6-6,L/L,1.0,NaN,13.50,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,John KwiatkowskiBoston College
19,30,JD Ogden,Sr,P,6-5,B/L,1.0,NaN,81.00,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD OgdenBoston College
26,50,Joey Ryan,Sr,P,6-5,L/R,1.0,NaN,0.00,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Joey RyanBoston College
32,47,Matthew Spada,Jr,P,6-1,L/L,1.0,NaN,0.00,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matthew SpadaBoston College
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74425,32,Maxwell Kasprowisz,Sr,P,6-0,-/L,1.0,NaN,18.00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maxwell KasprowiszWilliam Peace
74434,35,Jarrett Hall,Sr,P,5-10,-/R,1.0,1.0,23.63,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jarrett HallWilliam Peace
74443,2,Eddie Sanchez,Jr,SS,5-10,NaN,1.0,NaN,13.50,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eddie SanchezWilliam Peace
74452,52,Landon Coleman,Fr,P,6-1,-/R,1.0,NaN,0.00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Landon ColemanWilliam Peace


In [31]:
# Filter pitching_df based on Player_School in UVNRoster
pitching_dfUVN = pitching_df[pitching_df['Player_School'].isin(UVNROSTER['Player_School'])]

# Filter hitting_df based on Player_School in UVNRoster
hitting_dfUVN = hitting_df[hitting_df['Player_School'].isin(UVNROSTER['Player_School'])]
pitching_dfUVN

,#,Player,Yr,Pos,Ht,B/T,App,GS,ERA,IP,...,1,2,3,4,5,6,7,8,9,Player_School
3278,48,Brendan Walker,Jr,P,6-0,R/R,1.0,NaN,0.00,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brendan WalkerStetson
4844,30,Ryan Bilka,Jr,P,6-1,R/R,1.0,NaN,1.93,4.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ryan BilkaRichmond
7743,34,Connor Byrne,So,P,6-2,R/R,1.0,NaN,0.00,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Connor ByrneSeton Hall
8079,2,JP Podgorski,Fr,P,6-1,R/R,1.0,1.0,45.00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JP PodgorskiVillanova
8700,44,Charlie Jones,So,P,6-3,-/R,1.0,NaN,0.00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charlie JonesHigh Point
13563,43,Nick Tamburro,Sr,P,6-2,R/R,1.0,1.0,3.00,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nick TamburroMonmouth
15044,52,Owen Schexnaydre,Jr,P,6-5,R/R,1.0,1.0,4.50,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Owen SchexnaydreLouisiana Tech
15105,45,Jacob Havern,Jr,P,6-6,R/R,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jacob HavernLouisiana Tech
16655,6,Brendan Callanan,So,UT,6-0,R/R,1.0,1.0,12.46,4.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brendan CallananIona
16871,45,Ethan Marzo,So,P,6-0,R/R,1.0,NaN,20.25,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ethan MarzoMarist


In [32]:
hitting_dfUVN

,#,Player,Yr,Pos,Ht,B/T,GP,GS,BA,OBPct,...,1,2,3,4,5,6,7,8,9,Player_School
2697,51,Ryan Preisano,So,INF,6-4,NaN,4,2,0.500,0.667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ryan PreisanoWake Forest
4552,48,Brendan Walker,Jr,P,6-0,R/R,1,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brendan WalkerStetson
6615,4,Jack Arcamone,So,C,6-1,NaN,3,3,0.111,0.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jack ArcamoneRichmond
6635,33,Aidan O'Keefe,So,INF,5-10,NaN,2,1,0.400,0.400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aidan O'KeefeRichmond
6671,30,Ryan Bilka,Jr,P,6-1,R/R,1,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ryan BilkaRichmond
10593,34,Connor Byrne,So,P,6-2,R/R,1,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Connor ByrneSeton Hall
11032,2,JP Podgorski,Fr,P,6-1,R/R,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JP PodgorskiVillanova
11790,44,Charlie Jones,So,P,6-3,-/R,1,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charlie JonesHigh Point
13952,7,Jack Spanier,So,INF,6-0,R/R,3,3,0.273,0.385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jack SpanierMinnesota
13965,15,Drew Berkland,Jr,UT,6-0,R/R,3,3,0.385,0.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drew BerklandMinnesota


In [33]:
hitting_dfUVN.to_csv('UVNHItting.csv')
pitching_dfUVN.to_csv('UVNPitching.csv')